# Load CATMAID Skeletons or Synapses

In [4]:
import sys
sys.path.append("../catmaid")
from catmaidhelper import CatmaidHelper

PID = 283

with open("/home/tommytang111/gap-junction-segmentation/secret.txt") as f:
    api_token = f.read()
url = 'https://zhencatmaid.com/'

catmaid = CatmaidHelper(url, api_token)
catmaid.set_project(PID)

skeletons = catmaid.get_skeletons()
skeleton_names = catmaid.load_skeleton_names(skeletons)

In [34]:
import pandas as pd
pid = 283

connectors = catmaid.fetch(
    url = f"/{pid}/connectors/",
    method = "post",
    data = {
        "project_id": pid,
    }).json()['connectors']

cid = connectors[0][0]

connector_info = catmaid.fetch(
    url = f"/{pid}/connectors/{cid}/",
    method = "get",
    data = {
        "project_id": pid,
        "connector_id": cid
    }).json()

print(connectors[:5])
print(connector_info)

[[19669427, 36636.2, 19182.7, 5760.0, 5, 3, 3, 1680190730.41252, 1680190730.41254], [19669428, 36438.1, 19187.1, 6360.0, 5, 3, 3, 1680192006.12029, 1680192006.12032], [19669429, 36471.9, 19158.5, 6030.0, 5, 3, 3, 1680192753.17179, 1680192753.17182], [19669430, 35884.2, 18211.3, 7530.0, 5, 3, 3, 1680192902.78936, 1680192902.78936], [19669432, 36967.7, 17504.6, 10980.0, 5, 3, 3, 1680193083.14941, 1713053312.32528]]
{'connector_id': 19669427, 'x': 36636.2, 'y': 19182.7, 'z': 5760.0, 'confidence': 5, 'partners': [{'link_id': 3041631, 'partner_id': 19589120, 'confidence': 5, 'skeleton_id': 3038097, 'relation_id': 3033737, 'relation_name': 'gapjunction_with'}, {'link_id': 3041630, 'partner_id': 19499155, 'confidence': 5, 'skeleton_id': 3034362, 'relation_id': 3033737, 'relation_name': 'gapjunction_with'}]}


In [33]:
skeleton_names['3038097']

'[ADAR]'

In [3]:
from GetNodelist import get_bounded_nodes
from tqdm import tqdm
import concurrent.futures as futures

# get all bounded node info

bounded_nodes = {}

with futures.ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(
        get_bounded_nodes,
        [catmaid]*len(skeletons),
        skeletons), total=len(skeletons)))

for i in range(len(results)):
    if not results[i]:
        continue
    bounded_nodes[skeletons[i]] = results[i]['nodelist']

100%|██████████| 547/547 [00:24<00:00, 22.56it/s]


In [ ]:
import json

with open('bounded_nodes_adult.json', 'w') as fp:
    json.dump(bounded_nodes, fp)

In [ ]:
import pandas as pd
from tqdm import tqdm
import concurrent.futures as futures

# Load all connector info

col_names = ["connector_id","x","y","z"]
col_ids = [0,1,2,3]

connectors = pd.DataFrame(catmaid.get_connectors()).iloc[:,col_ids]
connectors = connectors.rename(columns=dict(zip(col_ids,col_names)))

def get_link_info(connector_id):
    connector_subtable = []
    connector_info = catmaid.get_connector_info(connector_id)
    pre = ""
    pre_node = 0
    post = []
    post_nodes = []
    for partner in connector_info['partners']:
        if partner['relation_name'] == 'presynaptic_to':
            if partner['confidence'] < 5:
                continue
            pre = partner['skeleton_id']
            pre_node = partner['partner_id']
        if partner['relation_name'] == 'postsynaptic_to':
            if partner['confidence'] < 5:
                continue
            post.append(partner['skeleton_id'])
            post_nodes.append(partner['partner_id'])
    for i in range(len(post)):
        connector_subtable.append({
            "connector_id": connector_info['connector_id'],
            "x": connector_info['x'],
            "y": connector_info['y'],
            "z": connector_info['z'],
            "pre": pre,
            "pre_node": pre_node,
            "post": post[i],
            "post_node": post_nodes[i]})
    return connector_subtable

with futures.ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(get_link_info, connectors['connector_id']), total=len(connectors)))

connector_table = [item for sublist in results if sublist for item in sublist]
connector_table = pd.DataFrame(connector_table)

# only include elements within bounded nodes for each particular skeleton
pre_skel_filt = connector_table['pre'].isin(bounded_nodes.keys())
connector_table = connector_table[pre_skel_filt]

all_bounded_nodes = set()
for nodes in bounded_nodes.values():
    all_bounded_nodes.update(nodes)

connector_table_filt = connector_table[connector_table['pre_node'].isin(all_bounded_nodes)]

100%|██████████| 4455/4455 [02:50<00:00, 26.12it/s]


In [ ]:
connector_table_filt = connector_table[connector_table['pre_node'].isin(all_bounded_nodes)]

# rename pre and post skeletons
connector_table_filt.loc[:,'pre'] = connector_table_filt['pre'].astype(str)
connector_table_filt.loc[:,'pre'] = connector_table_filt['pre'].map(skeleton_names)
connector_table_filt.loc[:,'post'] = connector_table_filt['post'].astype(str)
connector_table_filt.loc[:,'post'] = connector_table_filt['post'].map(skeleton_names)

/tmp/ipython-input-3003278493.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1854528' '1854405' '1857842' ... '1857421' '1853668' '1853519']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  connector_table_filt.loc[:,'post'] = connector_table_filt['post'].astype(str)


In [ ]:
import re

def clean_text(s):
    """ Remove non-alphanumeric, non short-dash from text
    """
    return re.sub(r'[^a-zA-Z0-9-]+', '', s)

connector_table_filt.loc[:,'pre'] = connector_table_filt['pre'].apply(clean_text)
connector_table_filt.loc[:,'post'] = connector_table_filt['post'].apply(clean_text)

connector_table_filt.to_csv("connector_table_adult.csv")

In [ ]:
import pandas as pd
from neuron_info import npair

name_fixes = {
    'PVNLorR1': 'PVWL',
    'PVNLorR2': 'PVWL',
    'PVNLorR3': 'PVWR',
}
name_remove = [
    'glia','fragment','musclefragment',
    'basementmembrane','hyp'
]

connector_table = pd.read_csv("connector_table_dauer-2.csv", index_col=0)
connector_table = connector_table[~connector_table['post'].isin(name_remove)]
connector_table.loc[:,'pre'] = connector_table['pre'].replace(name_fixes)
connector_table.loc[:,'post'] = connector_table['post'].replace(name_fixes)

# load existing classifications
classifications = pd.read_csv("consensus_classification.csv")
classifications['edge'] = classifications['pre'] + '-' + classifications['post']

In [ ]:
edge_classes = []
for edge in connector_table.loc[:,['pre','post']].map(npair).values:
    edge_name = edge[0] + '-' + edge[1]
    if edge_name not in classifications['edge'].values:
        edge_class = 'other'
    else:
        edge_class = classifications[classifications['edge'] == edge_name]['Classification'].values[0]
    edge_classes.append(edge_class)

In [ ]:
connector_table['classification'] = edge_classes

connector_table.to_csv("connector_table_dauer-2.csv")